In [1]:
import pandas as pd
import numpy as np
print(pd.__version__)

0.20.3


## 使用Datetime数据节省时间

In [27]:
df = pd.read_csv('test1.csv')
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 0:10,0.580
2,1/1/13 0:20,0.572
3,1/1/13 0:30,0.596
4,1/1/13 0:40,0.592


In [22]:
#如果没有特殊声明，那么自定义的date_time将会使用一个 object 的dtype类型，作为 str 类型会极大的影响效率
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [23]:
type(df.iat[0,0])#基于索引位置的选择方法

str

[pandas.to_datetime用法](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)

In [24]:
#格式化为datetime对象数组（pandas称之为时间戳）
df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].dtype

dtype('<M8[ns]')

In [25]:
#转化后的效果
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 00:10:00,0.580
2,2013-01-01 00:20:00,0.572
3,2013-01-01 00:30:00,0.596
4,2013-01-01 00:40:00,0.592


In [41]:
import time
def timeit(fun):
    def wrapper(*args,**kwargs):
        t1=time.time()
        res = fun(*args,**kwargs)
        t2 = time.time()
        grot = t2-t1
        print("所需时间：%.6f" %grot)
        
        return res
    return wrapper

#通过 timeit 装饰器来测试一下上面代码的转化时间
df = pd.read_csv('test1.csv')
@timeit
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

df['date_time'] = convert(df, 'date_time')

所需时间：0.001028


In [43]:
#更快方法--设置了转化的格式format
#由于在CSV中的datetimes并不是 ISO 8601 格式的，如果不进行设置的话，那么pandas将使用 dateutil 包把每个字符串str转化成date日期
#相反，如果原始数据datetime已经是 ISO 8601 格式了，那么pandas就可以立即使用最快速的方法来解析日期
df = pd.read_csv('test1.csv')

@timeit
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name], format='%d/%m/%y %H:%M')

df['date_time'] = convert_with_format(df, 'date_time')

所需时间：0.000995


## Pandas数据的循环操作

In [44]:
# 使用apply方法写一个函数，函数里面写好时间条件的逻辑代码
def apply_tariff(kwh, hour):
    """计算每个小时的电费"""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

In [50]:
# 最烂方法
df = pd.read_csv('test1.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')
@timeit
def apply_tariff_loop(df):
    """Calculate costs in loop.  Modifies `df` inplace."""
    energy_cost_list = []
    for i in range(len(df)):
        # 获取用电量和时间（小时）
        energy_used = df.iloc[i]['energy_kwh']
        hour = df.iloc[i]['date_time'].hour
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list
    
apply_tariff_loop(df)

所需时间：0.002992


In [51]:
# 常用方法 apply()
# lambda函数将帮助你将两列数据传递给apply_tariff()
df = pd.read_csv('test1.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')
@timeit
def apply_tariff_withapply(df):
    df['cost_cents'] = df.apply(
    lambda row: apply_tariff(
        kwh=row['energy_kwh'],
        hour=row['date_time'].hour),
    axis=1)
    
apply_tariff_withapply(df)

所需时间：0.004990


In [52]:
# 推荐做法使用itertuples() 和iterrows() 循环
df = pd.read_csv('test1.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')
@timeit
def apply_tariff_iterrows(df):
    energy_cost_list = []
    for index, row in df.iterrows():
        # 获取用电量和时间（小时）
        energy_used = row['energy_kwh']
        hour = row['date_time'].hour
        # 添加cost列表
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list

apply_tariff_iterrows(df)

所需时间：0.000997


In [54]:
# 矢量化操作：使用.isin()选择数据
df = pd.read_csv('test1.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')
df.set_index('date_time', inplace=True)

@timeit
def apply_tariff_isin(df):
    # 定义小时范围Boolean数组
    peak_hours = df.index.hour.isin(range(17, 24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    off_peak_hours = df.index.hour.isin(range(0, 7))

    # 使用上面的定义
    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12

apply_tariff_isin(df)

所需时间：0.059863


In [57]:
# 还可以做的更好,可以使用Pandas的pd.cut() 函数
#完全矢量化的方式： pd.cut() 根据每小时所属的bin应用一组标签(costs)
#注意include_lowest参数表示第一个间隔是否应该是包含左边的(您希望在组中包含时间= 0)
df = pd.read_csv('test1.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')
df.set_index('date_time', inplace=True)
@timeit
def apply_tariff_cut(df):
    cents_per_kwh = pd.cut(x=df.index.hour,
                           bins=[0, 7, 17, 24],
                           include_lowest=True,
                           labels=[12, 20, 28]).astype(int)
    df['cost_cents'] = cents_per_kwh * df['energy_kwh']
apply_tariff_cut(df)

所需时间：0.036932
